```
cd /Users/dorislee/Desktop/Fall2016/Research/seg/crowd-seg/analysis
bash herokuDBupdate.sh
```

In [1]:
cd ../web-app/

/Users/dorislee/Desktop/Fall2016/Research/seg/crowd-seg/web-app


In [2]:
%pylab inline 
%load_ext autoreload
%autoreload 2
from analysis_toolbox import *
import ast

Populating the interactive namespace from numpy and matplotlib


In [3]:
save_db_as_csv()
img_info,object_tbl,bb_info,hit_info = load_info()

/Users/dorislee/Desktop/Fall2016/Research/seg/data


/Users/dorislee/.virtualenvs/py27dev/lib/python2.7/site-packages/pandas/io/parsers.py:648: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.
  ParserWarning)


Add a new ``status`` column to hit_info

In [97]:
hit_info['status'] = pd.Series(['undetermined' for _i in range(len(hit_info))], index=hit_info.index)

In [4]:
cd ../crowd-seg/web-app/

/Users/dorislee/Desktop/Fall2016/Research/seg/crowd-seg/web-app


In [111]:
from boto.mturk.connection import MTurkRequestError
from boto.mturk.connection import MTurkConnection
from secret import SECRET_KEY,ACCESS_KEY
DEV_ENVIROMENT_BOOLEAN = True
#This allows us to specify whether we are pushing to the sandbox or live site.
if DEV_ENVIROMENT_BOOLEAN:
    AMAZON_HOST = 'mechanicalturk.sandbox.amazonaws.com'
    MastersQualID = '2F1KVCNHMVHV8E9PBUB2A4J79LU20F'
else:
    AMAZON_HOST = 'mechanicalturk.amazonaws.com'
    MastersQualID = '2NDP2L92HECWY8NS8H3CK0CP5L9GHO'

#Start Configuration Variables
AWS_ACCESS_KEY_ID = ACCESS_KEY
AWS_SECRET_ACCESS_KEY = SECRET_KEY

connection = MTurkConnection(aws_access_key_id=AWS_ACCESS_KEY_ID,
							 aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
							 host=AMAZON_HOST)


In [139]:
all_hits = [hit for hit in connection.get_all_hits()]
error_log = open("error.log",'w')
object_tbl['approvedBB_count'] = pd.Series(np.zeros(len(object_tbl)), index=object_tbl.index)

In [142]:
for hit in all_hits:
    assignments = connection.get_assignments(hit.HITId)
    for assignment in assignments:
        task = hit_info[hit_info.hit_id==assignment.HITId]
        if len(task)>0:
            actions = ast.literal_eval(task.actions.get_values()[0])
            numClicks = actions.count("addClick")
            try: 
                if numClicks >3:
                    hit_info = hit_info.set_value(task.index[0],'status',"approved")
                    #Add additional counts to object table 
                    object_tbl = object_tbl.set_value(task.object_id,'approvedBB_count', \
                    object_tbl._iloc[task.object_id].approvedBB_count.get_values()[0]+1)                    
                    connection.approve_assignment(assignment.AssignmentId)
                else:
                    hit_info = hit_info.set_value(task.index[0],'status',"rejected")
                    connection.reject_assignment(assignment.AssignmentId)
            except:#(MTurkRequestError):
                print "Problem with rejecting/approving: ", assignment.AssignmentId
                error_log.write(assignment.AssignmentId+'\n')
error_log.close()

Problem with rejecting/approving:  3ZAK8W07I56NAZT186Z0R766NVD0UR
Problem with rejecting/approving:  3XXU1SWE8NN3VQF321I9R6UCGSL0AF
Problem with rejecting/approving:  3G5F9DBFOQPYYEOXB4GQTL76NJNVHC
Problem with rejecting/approving:  3AUQQEL7U6LNNEI1DW95NT8P12M0VG
Problem with rejecting/approving:  3DYGAII7PM0Y6NQ48IULW7APSIIPQH
Problem with rejecting/approving:  33LKR6A5KFC8HBZU4KDQ0G3E99BT1V
Problem with rejecting/approving:  3FPRZHYEPZZJ46HYWJROPR6W4E5V35
Problem with rejecting/approving:  34Z02EIMIT58AMEC819QMBK44H60T0
Problem with rejecting/approving:  34Z02EIMIT58AMEC819QMBK44H5T0S
Problem with rejecting/approving:  31IBVUNM9TREKIXRGESQQ0VT74ZFVN


In [144]:
object_tbl.to_csv('../../data/object_tbl_mturk.csv')

In [145]:
hit_info.to_csv('../../data/hit_mturk.csv')

In [ ]:
# if assignment.HITId in list(hit_info.hit_id):
#     hit_info.hit_id==assignment.HITId